In [21]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("../TinkerEmulator/")
import TinkerEmulator
%matplotlib inline
%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
np.log(7.38376617e-04)

-7.211056542395139

In [23]:
emu = TinkerEmulator.TinkerEmulator()

(7,)
[-7.21105654 -4.95066548 -1.56694353 -4.10424403 -2.99017707  1.35357591
 -0.71213767]
[7.38376617e-04 7.07869664e-03 2.08682040e-01 1.65024894e-02
 5.02785334e-02 3.87124403e+00 4.90594346e-01]
0 [-7.44791527 -7.46475535 -2.59955668 -5.88271629 -3.49390804 22.35883024
  2.0321575 ]
[-7.21105654 -4.95066548 -1.56694353 -4.10424403 -2.99017707  1.35357591
 -0.71213767]
[7.38376617e-04 7.07869664e-03 2.08682040e-01 1.65024894e-02
 5.02785334e-02 3.87124403e+00 4.90594346e-01]
1 [-8.23942986 13.07829233 -1.64534545 24.55006208 -2.20850298  6.52984174
  2.70231684]
[-7.21105654 -4.95066548 -1.56694353 -4.10424403 -2.99017707  1.35357591
 -0.71213767]
[7.38376617e-04 7.07869664e-03 2.08682040e-01 1.65024894e-02
 5.02785334e-02 3.87124403e+00 4.90594346e-01]
2 [-2.39901555 -4.98489679  0.62077564 27.23396802  6.66307742  6.46055102
  5.09675166]
[-7.21105654 -4.95066548 -1.56694353 -4.10424403 -2.99017707  1.35357591
 -0.71213767]
[7.38376617e-04 7.07869664e-03 2.08682040e-01 1.65024894

In [24]:
cosmology = [0.0043*0.7**2, 0.286*0.7**2, -1, 1, 3.0, 70, 3.0]#Obh2 Och2 w0 n_s ln10As H0 N_eff
emu.predict_tinker_parameters(cosmology, 0.2)

(4.468028611188585,
 0.28448480666666665,
 1.4057166042127367,
 1.3069858166666666,
 0.6898978333333333,
 -1.2134163872509278)

In [25]:
cosmo_pars = np.loadtxt("../data/parameters.txt")
means = np.loadtxt("../data/rotated_means.txt")
variances = np.loadtxt("../data/rotated_variances.txt")
rotation_matrix = np.loadtxt("../data/rotation_matrix.txt")

In [26]:
y =  means-np.mean(means, axis=0)
ymean = np.mean(means, axis=0)
np.savetxt("../data/y.txt", y)
np.savetxt("../data/ymean.txt", ymean)

In [27]:
rotation_matrix.shape

(4, 4)

In [28]:
for item in np.array(emu.lamb):
    print("    {"+",".join([str(i) for i in item])+"},")

    {-7.447915268015301,-7.464755353609274,-2.599556684860715,-5.882716288518414,-3.4939080354809993,22.358830237008306,2.032157502031422},
    {-8.239429860610695,13.07829233346521,-1.6453454526929423,24.550062077519733,-2.208502981716708,6.529841736884619,2.702316841472215},
    {-2.3990155505181465,-4.984896790635661,0.620775644293421,27.233968017539855,6.663077418826718,6.4605510210300094,5.096751656964856},
    {-4.200230133291322,-3.9710113321459435,2.1024385590628487,23.33257434503111,5.148802333947621,10.753336183413882,9.909683275874231},


In [29]:
print(cosmo_pars.shape, means.shape, variances.shape, rotation_matrix.shape)

(40, 7) (40, 4) (40, 4) (4, 4)


In [32]:
def kernel(x,xprime, lamb):
    """
    lamb : d,1
    xprime : n,d
    x m,d
    return 40,1
    """
    result = np.zeros((len(x), len(xprime)))
    for i in range(len(x)):
        for j in range(len(xprime)):
            result[i,j] = np.exp(-1*np.sum((x[i]-xprime[j])**2/(2*(lamb.reshape(-1,1).T)), axis=1))
    return result
def sigma(x, xprime, lamb):
    return kernel(x,xprime, lamb) 

In [33]:
x0 = np.array([0.0043*0.7**2, 0.286*0.7**2, -1, 1, 3.0, 70, 3.0]).reshape(1,-1)
x1 = np.array([0.0043*0.7**2, 0.286*0.7**2, -1, 1, 3.0, 70, 5.0]).reshape(1,-1)
print(emu.GP_list[0].get_matrix(x0,x1))
print(sigma(x0, x1, np.exp(np.array(emu.lamb)[0])))

[[0.76943028]]
[[0.76943028]]


In [81]:
emu.lamb

[array([-7.44791527, -7.46475535, -2.59955668, -5.88271629, -3.49390804,
        22.35883024,  2.0321575 ]),
 array([-8.23942986, 13.07829233, -1.64534545, 24.55006208, -2.20850298,
         6.52984174,  2.70231684]),
 array([-2.39901555, -4.98489679,  0.62077564, 27.23396802,  6.66307742,
         6.46055102,  5.09675166]),
 array([-4.20023013, -3.97101133,  2.10243856, 23.33257435,  5.14880233,
        10.75333618,  9.90968328])]

In [34]:
print(",".join([str(item)for item in [0.0043*0.7**2, 0.286*0.7**2, -1, 1, 3.0, 70, 5.0]]))

0.002107,0.14013999999999996,-1,1,3.0,70,5.0


In [94]:
def prediction(xprime):
    print(sigma(xprime.T,cosmo_pars,np.exp(np.array(emu.lamb))[0])[0][0])
    result = []
    for i in range(len(emu.lamb)):
        mu = np.mean(means[:,i])
        sigma_xprime_x = sigma(xprime.T,cosmo_pars,np.exp(np.array(emu.lamb))[i])
        print(sigma_xprime_x.shape)
        sigma_x_x = sigma(cosmo_pars,cosmo_pars, np.exp(np.array(emu.lamb)[i]))+variances[:,i]*np.eye(len(cosmo_pars))
        if i==0:
            print("test")
            print(np.linalg.inv(sigma_x_x)[0], (means[:,i]-mu), np.linalg.inv(sigma_x_x).dot(means[:,i]-mu)[0], sigma_xprime_x[0][0])
            print("#####"*10)
        result.append(mu+sigma_xprime_x.dot(np.linalg.inv(sigma_x_x).dot(means[:,i]-mu)))
        if i==0:
            print(((sigma_x_x)[10,10]))
            print((np.linalg.inv(sigma_x_x)[20,10]))
    print(sigma_x_x.shape)
    print(result, np.mean(means, axis=0))
    return np.dot(rotation_matrix,np.array(result)).flatten()

In [95]:
(means[:,0]-np.mean(means[:,0]))[18]

0.37515414999999996

In [96]:
prediction(np.array([0.0043*0.7**2, 0.286*0.7**2, -1, 1, 3.0, 70, 3.0]).reshape(-1,1))#

0.23867028218751893
(1, 40)
test
[ 5.49668965e+01  8.03580384e-01 -5.80580295e+00  5.72987755e+00
 -1.06162897e+01  3.63960075e-01 -1.14902105e+01 -5.96920925e-01
  5.52670068e-01 -9.21030066e+00  1.07538299e+00 -4.29068099e+00
 -1.87661912e+01 -3.83283219e+00  2.56806517e-01 -2.00247690e+00
  1.61000644e+00  1.82314949e-01 -2.92951514e+00  3.93692966e+00
 -3.53007529e+00 -1.58272169e+01 -5.26731023e+00  3.55959449e+00
  5.48738918e+00  9.92022064e+00 -5.19240972e-01  1.39492729e-02
 -4.72991336e-01  1.62473965e+00 -2.30778852e+00  8.43129140e-01
  2.97261223e+00 -1.99635320e+00  2.06215277e+00  4.43740060e-01
  2.06942155e+00 -8.72816470e+00  3.05497529e+00  5.30030886e+00] [ 0.22772415 -0.28278585  0.32128415  0.17846415  0.40985415 -0.76878285
  0.40140415  0.24942415 -0.39614285  0.31643415 -0.24992585 -0.12223585
  0.26071415  0.08741415 -0.76097985 -0.06336585 -0.09263585 -0.06395585
  0.37515415  0.37989415  0.04260415  0.15116415 -0.46622185  0.97937415
  0.57734415 -0.09004585

array([ 1.16115019, -0.59625504,  0.55550433,  0.73369923])

In [37]:
emu.emulate(np.array([0.0043*0.7**2, 0.286*0.7**2, -1, 1, 3.0, 70, 3.0]))

array([ 1.16115019, -0.59625504,  0.55550433,  0.73369923])

In [16]:
emu.predict_tinker_parameters(np.array([0.0043*0.7**2, 0.286*0.7**2, -1, 1, 3.0, 70, 3.0]), 0.3)

(4.432419359036935,
 0.32058653615384625,
 1.3586846020079848,
 1.347376928846154,
 0.728081326923077,
 -1.0947315122509278)

In [89]:
np.savetxt("../data/log_bias_metric.txt", np.array(emu.lamb))

In [120]:
np.array(emu.lamb).shape

(4, 7)